In [1]:
import requests, time
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
vehicle_urls = []


def fetch_page_and_convert_to_bs(url):

    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    
    # Delay 5 seconds before returning the soup object to prevent Python from sending too many requests in a rapid succession
    time.sleep(5)

    return soup


def get_vehicle_urls():
    
    counter = 1
    iterate = True
    pages = 0
        
    # Storing the base url the data will be fetched from
    base_url = 'https://www.cars.com/for-sale/searchresults.action/?dealerType=all&perPage=50&rd=10&searchSource=PAGINATION&sort=relevance&page='

    # Running a loop to determine how many pages with vehicle content exist
    while (iterate):
        
        """ 
            Creating a dynamic url that will update the page number in each pass of the loop
            until there is no vehicle content to be found
        """

        url = f'{base_url}{counter}'
        
        # Instantiating a beautiful soup object to store the html
        soup = fetch_page_and_convert_to_bs(url)        
        
        # Identify if any vehicles exist on the page
        has_vehicle = len(soup.find('a', class_ = 'shop-srp-listings__listing'))

        if (has_vehicle and pages < 3): # The "3" in the conditional to prevent the return of the entire cars.com list.
            pages += 1
            counter += 1
            
            # Find all the vehicles that exist on the page
            vehicles = soup.find_all('a', class_ = 'shop-srp-listings__listing')
            
            for vehicle in vehicles:
                vehicle_urls.append(vehicle['href'])
            
        else:
            iterate = False


In [3]:
get_vehicle_urls()
url=vehicle_urls
print(url)

['/vehicledetail/detail/771429339/overview/', '/vehicledetail/detail/782675683/overview/', '/vehicledetail/detail/780786906/overview/', '/vehicledetail/detail/782874496/overview/', '/vehicledetail/detail/759906922/overview/', '/vehicledetail/detail/780786905/overview/', '/vehicledetail/detail/778814930/overview/', '/vehicledetail/detail/782110566/overview/', '/vehicledetail/detail/774253853/overview/', '/vehicledetail/detail/778078127/overview/', '/vehicledetail/detail/782118329/overview/', '/vehicledetail/detail/775693455/overview/', '/vehicledetail/detail/778177902/overview/', '/vehicledetail/detail/771301441/overview/', '/vehicledetail/detail/772928320/overview/', '/vehicledetail/detail/782512548/overview/', '/vehicledetail/detail/778951004/overview/', '/vehicledetail/detail/772428590/overview/', '/vehicledetail/detail/767362885/overview/', '/vehicledetail/detail/781052664/overview/', '/vehicledetail/detail/775382273/overview/', '/vehicledetail/detail/762192909/overview/', '/vehicle

In [4]:
new_url=[]
for i in url:
    complete_list=(f'https://www.cars.com'+i)
    new_url.append(complete_list)
print(len(new_url))


150


In [5]:
# Now that the urls for each individual vehicle have been found use the fetch_page_and_convert_to_bs() function...
# each pass of the loop should scrape the pertinent data points from the page
# each scrape should store the data points as a dictionary

final_data=[]
counter = 0

def get_car_info(url):
        response = requests.get(url)
        soup = bs(response.text, 'html.parser')
        # Delay 5 seconds before returning the soup object to prevent Python from sending too many requests in a rapid succession    
        time.sleep(3)

        return soup  
    
    
for url in new_url:
    soup=get_car_info(url)
    
    #Confirms that the listing includes 
    try:
        year_make_model = soup.find('p', class_ = 'mmy-link').a.text
        year_make_model_list = year_make_model.split()
        list_len=len(year_make_model_list)
        #Evaluates that all required data elements have been supplied but checking the greatest index element.
        mileage = soup.find_all('li', class_ = 'vdp-details-basics__item')[10].span.text

    except:
        list_len=0

    
    if list_len==3:

        year=year_make_model_list[0]
        make=year_make_model_list[1]
        model=year_make_model_list[2]

        condition = soup.find('h1', class_ = 'vehicle-info__stock-type').text
        condition = condition.title()
        price = soup.find('span', class_='vehicle-info__price-display').text
        vin = soup.find_all('li', class_ = 'vdp-details-basics__item')[9].span.text
        mileage = soup.find_all('li', class_ = 'vdp-details-basics__item')[10].span.text
        transmission = soup.find_all('li', class_ = 'vdp-details-basics__item')[7].span.text
        engine = soup.find_all('li', class_ = 'vdp-details-basics__item')[8].span.text
        drive_type = soup.find_all('li', class_ = 'vdp-details-basics__item')[6].span.text
        fuel_type = soup.find_all('li', class_ = 'vdp-details-basics__item')[0].span.text
        exterior_color = soup.find_all('li', class_ = 'vdp-details-basics__item')[1].span.text
        interior_color = soup.find_all('li', class_ = 'vdp-details-basics__item')[3].span.text
        seller_name = soup.find('h3', class_ = 'vdp-dealer-info__title').text
        seller_address = soup.find('p', class_ = 'vdp-dealer-location__address').text
        
        #Parsing address field to extract zip code
        address_parts = seller_address.split()
        where_to_pull=len(address_parts)
        zip_code = address_parts[where_to_pull-1]
        
        vehicle = {
        'condition' : condition,
        'price' : price,
        'drive_type' : drive_type,
        'engine' : engine,
        'seller_name' : seller_name,
        'zip' : zip_code,
        'fuel_type' : fuel_type,
        'exterior_color' : exterior_color,
        'interior_color' : interior_color,
        'mileage' : mileage,
        'transmission' : transmission,
        'vin' : vin,
        'url' : url

        }
        print(condition, year, make, model)
        print(price)
        print(drive_type)
        print(engine)
        print(seller_name, zip_code)
        print(fuel_type)
        print(exterior_color)
        print(interior_color)
        print(mileage)
        print(transmission)
        print(vin)
        print(url)

    

New 2019 Ford F-150
$46,040
 RWD
 2.7L V6 24V PDI DOHC Twin Turbo
Randall Reed's Prestige Ford 75041
 Gasoline
 Oxford White
 Medium Earth Gray
 3
 Automatic
 1FTEW1CP7KKE31574
https://www.cars.com/vehicledetail/detail/782675683/overview/
1 __________________
New 2019 Ford F-150
$41,040
 RWD
 2.7L V6 24V PDI DOHC Twin Turbo
Randall Reed's Prestige Ford 75041
 Gasoline
 White
 Black
 3,095
 Automatic
 1FTEW1CP7KKC66657
https://www.cars.com/vehicledetail/detail/782874496/overview/
2 __________________
New 2020 Chevrolet Traverse
$35,405
 FWD
 3.6L V6 24V GDI DOHC
Jupiter Chevrolet Inc 75041
 Gasoline
 Silver Ice Metallic
 Jet Black
 5
 9-Speed Automatic
 1GNERGKW9LJ117197
https://www.cars.com/vehicledetail/detail/780786905/overview/
3 __________________
New 2019 Toyota Prius
$26,103
 FWD
 1.8L I4 16V MPFI DOHC Hybrid
Sport City Toyota 75228
 Hybrid
 Blizzard Pearl
 Beige
 Not provided
 Automatic CVT
 JTDKARFU3K3095949
https://www.cars.com/vehicledetail/detail/782110566/overview/
4 ______

New 2019 Ford Fusion
$25,770
 FWD
 1.5L I4 16V GDI DOHC Turbo
Randall Reed's Prestige Ford 75041
 Gasoline
 Ingot Silver Metallic
 Ebony
 2,496
 6-Speed Automatic
 3FA6P0HDXKR169769
https://www.cars.com/vehicledetail/detail/781303276/overview/
35 __________________
New 2019 Ford Fusion
$23,835
 FWD
 2.5L I4 16V MPFI DOHC
Randall Reed's Prestige Ford 75041
 Gasoline
 Oxford White
 Medium Light Stone
 2
 6-Speed Automatic
 3FA6P0G77KR282610
https://www.cars.com/vehicledetail/detail/782675666/overview/
36 __________________
New 2019 Ford Expedition
$73,385
 RWD
 3.5L V6 24V GDI DOHC Twin Turbo
Randall Reed's Prestige Ford 75041
 Gasoline
 Agate Black Metallic
 Ebony
 2,874
 10-Speed Automatic
 1FMJU1KT6KEA10913
https://www.cars.com/vehicledetail/detail/781303263/overview/
37 __________________
New 2019 Volkswagen Jetta
$17,504
 FWD
 1.4L I4 16V GDI DOHC Turbo
Rusty Wallis Volkswagen 75041
 Gasoline
 Pure White
 Titan Black
 5
 8-Speed Automatic
 3VWCB7BU2KM181888
https://www.cars.com/vehi

New 2019 GMC Acadia
$34,055
 FWD
 3.6L 6 cyl Fuel Injected
Heritage Buick GMC 75087
 Gasoline
 Quicksilver Metallic
 Dark Ash Gray/ Light Ash Gray
 4,113
 Not Specified
 1GKKNLLS1KZ155474
https://www.cars.com/vehicledetail/detail/751806959/overview/
68 __________________
New 2019 GMC Acadia
$28,330
 FWD
 2.5L 4 cyl Fuel Injected
Heritage Buick GMC 75087
 Gasoline
 Ebony Twilight Metallic
 Jet Black
 5
 Automatic
 1GKKNKLA4KZ223932
https://www.cars.com/vehicledetail/detail/765006544/overview/
69 __________________
New 2019 GMC Acadia
$37,870
 FWD
 3.6L 6 cyl Fuel Injected
Heritage Buick GMC 75087
 Gasoline
 Ebony Twilight Metallic
 Jet Black
 11
 Automatic
 1GKKNMLS8KZ257375
https://www.cars.com/vehicledetail/detail/770468335/overview/
70 __________________
New 2019 Toyota Camry
$25,188
 FWD
 2.5L I4 16V PDI DOHC
Sport City Toyota 75228
 Gasoline
 Red
 Black
 Not provided
 8-Speed Automatic
 4T1B11HK3KU279534
https://www.cars.com/vehicledetail/detail/780403890/overview/
71 _____________

New 2019 Mazda Mazda3
$26,820
 AWD
 2.5L I4 16V GDI DOHC
Town North Mazda 75080
 Gasoline
 Machine Gray Metallic
 Beige
 12
 6-Speed Automatic
 JM1BPBDMXK1133097
https://www.cars.com/vehicledetail/detail/774058378/overview/
102 __________________
New 2019 Ford Escape
$25,542
 FWD
 1.5L I4 16V GDI DOHC Turbo
North Central Ford 75080
 Gasoline
 Magnetic Gray
 Gray Charcoal
 41
 6-Speed Automatic
 1FMCU0GD5KUB34567
https://www.cars.com/vehicledetail/detail/769473347/overview/
103 __________________
New 2019 Nissan Sentra
$15,988
 FWD
 1.8L I4 16V MPFI DOHC
Courtesy Nissan 75080
 Gasoline
 Cloud White
 Charcoal
 45
 Automatic CVT
 3N1AB7AP5KY319893
https://www.cars.com/vehicledetail/detail/775799324/overview/
104 __________________
New 2019 Volvo XC90
$68,075
 AWD
 2.0L I4 16V GDI DOHC
Crest Volvo Cars 75075
 Gasoline
 Crystal White Pearl
 Charcoal
 1
 8-Speed Automatic
 YV4A22PM1K1514212
https://www.cars.com/vehicledetail/detail/776161832/overview/
105 __________________
New 2019 Volvo XC